In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer,MinMaxScaler
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold,StratifiedKFold,cross_val_score,GridSearchCV,train_test_split

In [2]:
def read_files(Train,Test):
    titanic_train=pd.read_csv(Train,index_col=0)
    titanic_test=pd.read_csv(Test,index_col=0)
    return titanic_train,titanic_test

In [3]:
def fix_names(df_for_names):
    Titles_dict={'Mrs.':'Mrs.','Miss.':'Miss.','Master.':'Master.','Lady.':'Lady.','Dr.':'Sir.','Sir.':'Sir.','Major.':'Sir.',
             'Capt.':'Sir.','Col.':'Sir.','Mr.':'Mr.','Rev.':'Sir.','Don.':'Sir.','Countess.':'Lady',
             'Mlle.':'Miss.','Ms.':'Miss.','Mme.':'Mr.','Jonkheer.':'Mr.','Dona.':'Miss.'}
    for lab,row in df_for_names.iterrows():
        words=row['Name'].split()
        for word in words:
            if word in Titles_dict.keys():
                df_for_names.loc[lab,'Name']=Titles_dict[word]
    return df_for_names

In [4]:
def fix_tickets(df_tickets):
    Ticket_dict={'1':1,'2':2,'3':3,'A':4,'C':5,'P':6,'S':7}
    for lab,row in df_tickets.iterrows():
        if row['Ticket'][0] in Ticket_dict.keys():
            df_tickets.loc[lab,'Ticket']=Ticket_dict[row['Ticket'][0]]
        else:
            df_tickets.loc[lab,'Ticket']=0     
    return  df_tickets

In [5]:
def fix_cabin_with_similarities(df_cabin):
    for lab, row in df_cabin.iterrows():
        if pd.notna(row['Cabin']):
            df_cabin.loc[lab,'Cabin']=row['Cabin'][0]
    Cabin_df=df_cabin.groupby(['Cabin']).mean()
    Cabin_labels=list(Cabin_df.index)
    Cabin_Fare=Cabin_df[['Fare']]
    Cabin_Parch=Cabin_df[['Parch']]
    Cabin_Fare_np=Cabin_Fare.to_numpy()
    Cabin_Parch_np=Cabin_Parch.to_numpy()
    Cabin_Fare_Kmeans=KMeans(n_clusters=7).fit(Cabin_Fare_np)
    Cabin_Parch_Kmeans=KMeans(n_clusters=7).fit(Cabin_Parch_np)
    Cabin_Fare_Labels=list(Cabin_Fare_Kmeans.labels_)
    Cabin_Parch_Labels=list(Cabin_Parch_Kmeans.labels_)
    Cabin_Parch_dict=dict(zip(Cabin_Parch_Labels,Cabin_labels))
    Cabin_Fare_dict=dict(zip(Cabin_Fare_Labels,Cabin_labels))
    for lab, row in df_cabin.iterrows():
        if pd.isna(row['Cabin']):
            if pd.notna(row['Fare']):
                point=row[['Fare']].to_numpy()
                CB_key=Cabin_Fare_Kmeans.predict([point])[0]
                CB_Value=Cabin_Fare_dict[CB_key]
                df_cabin.loc[lab,'Cabin']=CB_Value
            else:
                point=row[['Parch']].to_numpy()
                CB_key=Cabin_Parch_Kmeans.predict([point])[0]
                CB_Value=Cabin_Parch_dict[CB_key]
                df_cabin.loc[lab,'Cabin']=CB_Value
    return df_cabin

In [6]:
def fix_cabin_with_most_frequent(df_cabin):
    for lab, row in df_cabin.iterrows():
        if pd.notna(row['Cabin']):
            df_cabin.loc[lab,'Cabin']=row['Cabin'][0]
    Max_Frequency=df_cabin['Cabin'].value_counts().idxmax()
    for lab, row in df_cabin.iterrows():
        if pd.isna(row['Cabin']):
            df_cabin.loc[lab,'Cabin']=Max_Frequency
    return df_cabin

In [7]:
def fix_age(df_age):
    for lab,row in df_age.iterrows():
        if pd.notna(row['Age']):
            if row['Age']<7:
                df_age.loc[lab,'Age']=0
            elif row['Age']>55:
                df_age.loc[lab,'Age']=2
            else:
                df_age.loc[lab,'Age']=1
        else:
            df_age.loc[lab,'Age']=3
    return df_age

In [8]:
def model_maker(n_neurons1,n_neurons2,optimizer='adam',initializer='glorot_uniform'):
    model=Sequential()
    model.add(Dense(units=n_neurons1,activation='relu',kernel_initializer=initializer,input_shape=(47,)))
    model.add(Dense(units=n_neurons2,activation='relu'))
    model.add(Dense(units=1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'],verbose=0)
    return model

In [9]:
titanic_train,titanic_test=read_files('train.csv','test.csv')
X0=fix_tickets(titanic_train)
T0=fix_tickets(titanic_test)
X1=fix_names(X0)
T1=fix_names(T0)
X2=fix_cabin_with_most_frequent(X1)
T2=fix_cabin_with_most_frequent(T1)
X3=X2
T3=T2

In [10]:
T3.loc[:,'Parch'].replace(9,6,inplace=True)

In [11]:
X3.loc[:,'Age'].fillna(0,inplace=True)
T3.loc[:,'Age'].fillna(0,inplace=True)

In [12]:
X3.loc[:,'Embarked'].fillna('S',inplace=True)
T3.loc[:,'Embarked'].fillna('S',inplace=True)

In [13]:
Train=X3.iloc[:,1:]
Test=T3
y=X3['Survived']

In [14]:
Train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,3,Mr.,male,22.0,1,0,4,7.2500,C,S
2,1,Mrs.,female,38.0,1,0,6,71.2833,C,C
3,3,Miss.,female,26.0,0,0,7,7.9250,C,S
4,1,Mrs.,female,35.0,1,0,1,53.1000,C,S
5,3,Mr.,male,35.0,0,0,3,8.0500,C,S


In [15]:
cs = MinMaxScaler()

In [16]:
Titanic_Columns=list(Train.columns)
print(Titanic_Columns)

['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


In [17]:
ContinousParameters=['Age','Fare']

In [18]:
CategoricalParameters=[member for member in Titanic_Columns if (member not in ContinousParameters and member!='Survived')]

In [19]:
TrainContinuous = cs.fit_transform(Train[ContinousParameters])
TestContinuous = cs.transform(Test[ContinousParameters])
One_Hot = OneHotEncoder().fit(Train[CategoricalParameters])
TrainCategorical = One_Hot.transform(Train[CategoricalParameters]).toarray()
TestCategorical = One_Hot.transform(Test[CategoricalParameters]).toarray()

In [20]:
TTrainX = np.hstack((TrainCategorical, TrainContinuous))
TTestX = np.hstack((TestCategorical, TestContinuous))

In [21]:
my_KerasClassifier=KerasClassifier(build_fn=model_maker,batch_size=10,epochs=100)

In [22]:
SKF=StratifiedKFold(n_splits=5,shuffle=True,random_state=10)

initializers=['glorot_uniform','uniform','normal']
epochss=[20,50]
batch_sizes=[10,20]
param_grids={'initializer':initializers,'epochs':epochss,'batch_size':batch_sizes}

In [23]:
N11=[12,24,36,48,64]
N22=[4,8,16,24]
param_grids={'n_neurons1':N11,'n_neurons2':N22}

In [24]:
print(param_grids)

{'n_neurons1': [12, 24, 36, 48, 64], 'n_neurons2': [4, 8, 16, 24]}


In [25]:
grid_search=GridSearchCV(estimator=my_KerasClassifier,param_grid=param_grids,cv=SKF)

In [26]:
results=grid_search.fit(TTrainX,y)

Train on 712 samples
Epoch 1/100
712/712 [==============================] - 1s 2ms/sample - loss: 0.6327 - accuracy: 0.6264
Epoch 2/100
712/712 [==============================] - 0s 140us/sample - loss: 0.5597 - accuracy: 0.7303
Epoch 3/100
712/712 [==============================] - 0s 147us/sample - loss: 0.5245 - accuracy: 0.7612
Epoch 4/100
712/712 [==============================] - 0s 139us/sample - loss: 0.4927 - accuracy: 0.7935
Epoch 5/100
712/712 [==============================] - 0s 133us/sample - loss: 0.4691 - accuracy: 0.7935
Epoch 6/100
712/712 [==============================] - 0s 132us/sample - loss: 0.4536 - accuracy: 0.8006
Epoch 7/100
712/712 [==============================] - 0s 142us/sample - loss: 0.4390 - accuracy: 0.8132
Epoch 8/100
712/712 [==============================] - 0s 146us/sample - loss: 0.4288 - accuracy: 0.8174
Epoch 9/100
712/712 [==============================] - 0s 135us/sample - loss: 0.4184 - accuracy: 0.8216
Epoch 10/100
712/712 [==============

712/712 [==============================] - 0s 143us/sample - loss: 0.2998 - accuracy: 0.8750
Epoch 79/100
712/712 [==============================] - 0s 129us/sample - loss: 0.2984 - accuracy: 0.8750
Epoch 80/100
712/712 [==============================] - 0s 143us/sample - loss: 0.2988 - accuracy: 0.8806
Epoch 81/100
712/712 [==============================] - 0s 157us/sample - loss: 0.2956 - accuracy: 0.8820
Epoch 82/100
712/712 [==============================] - 0s 150us/sample - loss: 0.2938 - accuracy: 0.8834
Epoch 83/100
712/712 [==============================] - 0s 128us/sample - loss: 0.2970 - accuracy: 0.8778
Epoch 84/100
712/712 [==============================] - 0s 126us/sample - loss: 0.2948 - accuracy: 0.8834
Epoch 85/100
712/712 [==============================] - 0s 126us/sample - loss: 0.2963 - accuracy: 0.8848
Epoch 86/100
712/712 [==============================] - 0s 125us/sample - loss: 0.2945 - accuracy: 0.8848
Epoch 87/100
712/712 [==============================] - 0s 

713/713 [==============================] - 0s 132us/sample - loss: 0.4899 - accuracy: 0.8163
Epoch 5/100
713/713 [==============================] - 0s 126us/sample - loss: 0.4502 - accuracy: 0.8317
Epoch 6/100
713/713 [==============================] - 0s 130us/sample - loss: 0.4244 - accuracy: 0.8359
Epoch 7/100
713/713 [==============================] - 0s 128us/sample - loss: 0.4115 - accuracy: 0.8499
Epoch 8/100
713/713 [==============================] - 0s 130us/sample - loss: 0.4022 - accuracy: 0.8485
Epoch 9/100
713/713 [==============================] - 0s 126us/sample - loss: 0.3962 - accuracy: 0.8513
Epoch 10/100
713/713 [==============================] - 0s 126us/sample - loss: 0.3902 - accuracy: 0.8541
Epoch 11/100
713/713 [==============================] - 0s 125us/sample - loss: 0.3870 - accuracy: 0.8541
Epoch 12/100
713/713 [==============================] - 0s 126us/sample - loss: 0.3828 - accuracy: 0.8527
Epoch 13/100
713/713 [==============================] - 0s 125us

713/713 [==============================] - 0s 175us/sample - loss: 0.3048 - accuracy: 0.8878
Epoch 82/100
713/713 [==============================] - 0s 140us/sample - loss: 0.3057 - accuracy: 0.8920
Epoch 83/100
713/713 [==============================] - 0s 139us/sample - loss: 0.3060 - accuracy: 0.8892
Epoch 84/100
713/713 [==============================] - 0s 135us/sample - loss: 0.3041 - accuracy: 0.8892
Epoch 85/100
713/713 [==============================] - 0s 123us/sample - loss: 0.3017 - accuracy: 0.8906
Epoch 86/100
713/713 [==============================] - 0s 128us/sample - loss: 0.3014 - accuracy: 0.8934
Epoch 87/100
713/713 [==============================] - 0s 126us/sample - loss: 0.3033 - accuracy: 0.8892
Epoch 88/100
713/713 [==============================] - 0s 126us/sample - loss: 0.2998 - accuracy: 0.8878
Epoch 89/100
713/713 [==============================] - 0s 126us/sample - loss: 0.3016 - accuracy: 0.8920
Epoch 90/100
713/713 [==============================] - ETA

Epoch 7/100
713/713 [==============================] - 0s 135us/sample - loss: 0.5247 - accuracy: 0.8317
Epoch 8/100
713/713 [==============================] - 0s 132us/sample - loss: 0.5163 - accuracy: 0.8331
Epoch 9/100
713/713 [==============================] - 0s 130us/sample - loss: 0.5029 - accuracy: 0.8317
Epoch 10/100
713/713 [==============================] - 0s 133us/sample - loss: 0.4934 - accuracy: 0.8359
Epoch 11/100
713/713 [==============================] - 0s 139us/sample - loss: 0.4837 - accuracy: 0.8331
Epoch 12/100
713/713 [==============================] - 0s 139us/sample - loss: 0.4758 - accuracy: 0.8331
Epoch 13/100
713/713 [==============================] - 0s 136us/sample - loss: 0.4684 - accuracy: 0.8457
Epoch 14/100
713/713 [==============================] - 0s 133us/sample - loss: 0.4604 - accuracy: 0.8457
Epoch 15/100
713/713 [==============================] - 0s 128us/sample - loss: 0.4534 - accuracy: 0.8471
Epoch 16/100
713/713 [===========================

...

713/713 [==============================] - 0s 125us/sample - loss: 0.2256 - accuracy: 0.9004
Epoch 79/100
713/713 [==============================] - 0s 125us/sample - loss: 0.2277 - accuracy: 0.9046
Epoch 80/100
713/713 [==============================] - 0s 128us/sample - loss: 0.2297 - accuracy: 0.9032
Epoch 81/100
713/713 [==============================] - 0s 126us/sample - loss: 0.2296 - accuracy: 0.8962
Epoch 82/100
713/713 [==============================] - 0s 126us/sample - loss: 0.2233 - accuracy: 0.9032
Epoch 83/100
713/713 [==============================] - 0s 129us/sample - loss: 0.2344 - accuracy: 0.9046
Epoch 84/100
713/713 [==============================] - 0s 128us/sample - loss: 0.2283 - accuracy: 0.9032
Epoch 85/100
713/713 [==============================] - 0s 126us/sample - loss: 0.2254 - accuracy: 0.9046
Epoch 86/100
713/713 [==============================] - 0s 129us/sample - loss: 0.2252 - accuracy: 0.9060
Epoch 87/100
713/713 [==============================] - 0s 

713/713 [==============================] - 0s 130us/sample - loss: 0.4000 - accuracy: 0.8415
Epoch 5/100
713/713 [==============================] - 0s 128us/sample - loss: 0.3877 - accuracy: 0.8429
Epoch 6/100
713/713 [==============================] - 0s 132us/sample - loss: 0.3735 - accuracy: 0.8373
Epoch 7/100
713/713 [==============================] - 0s 128us/sample - loss: 0.3669 - accuracy: 0.8527
Epoch 8/100
713/713 [==============================] - 0s 126us/sample - loss: 0.3621 - accuracy: 0.8485
Epoch 9/100
713/713 [==============================] - 0s 126us/sample - loss: 0.3521 - accuracy: 0.8612
Epoch 10/100
713/713 [==============================] - 0s 128us/sample - loss: 0.3458 - accuracy: 0.8654
Epoch 11/100
713/713 [==============================] - 0s 126us/sample - loss: 0.3437 - accuracy: 0.8626
Epoch 12/100
713/713 [==============================] - 0s 128us/sample - loss: 0.3359 - accuracy: 0.8724
Epoch 13/100
713/713 [==============================] - 0s 128us

713/713 [==============================] - 0s 140us/sample - loss: 0.2399 - accuracy: 0.8976
Epoch 82/100
713/713 [==============================] - 0s 139us/sample - loss: 0.2338 - accuracy: 0.8920
Epoch 83/100
713/713 [==============================] - 0s 140us/sample - loss: 0.2326 - accuracy: 0.8934
Epoch 84/100
713/713 [==============================] - 0s 140us/sample - loss: 0.2374 - accuracy: 0.8906
Epoch 85/100
713/713 [==============================] - 0s 143us/sample - loss: 0.2369 - accuracy: 0.8906
Epoch 86/100
713/713 [==============================] - 0s 144us/sample - loss: 0.2325 - accuracy: 0.8934
Epoch 87/100
713/713 [==============================] - 0s 145us/sample - loss: 0.2302 - accuracy: 0.8976
Epoch 88/100
713/713 [==============================] - 0s 135us/sample - loss: 0.2351 - accuracy: 0.9004
Epoch 89/100
713/713 [==============================] - 0s 128us/sample - loss: 0.2317 - accuracy: 0.8962
Epoch 90/100
713/713 [==============================] - 0s 

891/891 [==============================] - 0s 141us/sample - loss: 0.3766 - accuracy: 0.8519
Epoch 8/100
891/891 [==============================] - 0s 136us/sample - loss: 0.3724 - accuracy: 0.8429
Epoch 9/100
891/891 [==============================] - 0s 138us/sample - loss: 0.3647 - accuracy: 0.8541
Epoch 10/100
891/891 [==============================] - 0s 145us/sample - loss: 0.3583 - accuracy: 0.8485
Epoch 11/100
891/891 [==============================] - 0s 136us/sample - loss: 0.3581 - accuracy: 0.8597
Epoch 12/100
891/891 [==============================] - 0s 137us/sample - loss: 0.3491 - accuracy: 0.8575
Epoch 13/100
891/891 [==============================] - 0s 135us/sample - loss: 0.3466 - accuracy: 0.8586
Epoch 14/100
891/891 [==============================] - 0s 135us/sample - loss: 0.3416 - accuracy: 0.8642
Epoch 15/100
891/891 [==============================] - 0s 149us/sample - loss: 0.3408 - accuracy: 0.8687
Epoch 16/100
891/891 [==============================] - 0s 14

891/891 [==============================] - 0s 141us/sample - loss: 0.2540 - accuracy: 0.8923
Epoch 84/100
891/891 [==============================] - 0s 168us/sample - loss: 0.2523 - accuracy: 0.8866
Epoch 85/100
891/891 [==============================] - 0s 149us/sample - loss: 0.2489 - accuracy: 0.9001
Epoch 86/100
891/891 [==============================] - 0s 126us/sample - loss: 0.2507 - accuracy: 0.8990
Epoch 87/100
891/891 [==============================] - 0s 123us/sample - loss: 0.2490 - accuracy: 0.8956
Epoch 88/100
891/891 [==============================] - 0s 127us/sample - loss: 0.2480 - accuracy: 0.8923
Epoch 89/100
891/891 [==============================] - 0s 130us/sample - loss: 0.2518 - accuracy: 0.8979
Epoch 90/100
891/891 [==============================] - 0s 134us/sample - loss: 0.2459 - accuracy: 0.8923
Epoch 91/100
891/891 [==============================] - 0s 128us/sample - loss: 0.2455 - accuracy: 0.8990
Epoch 92/100
891/891 [==============================] - 0s 

In [27]:
results.cv_results_

{'mean_fit_time': array([11.13621979, 10.64352579, 10.66141367, 10.16803098, 10.12696967,
        10.22614326, 10.25118165, 10.5418623 , 10.42211614, 10.38165836,
        10.41211901, 10.42768316, 10.63858438, 10.86544633, 11.36531591,
        11.04844441, 11.66044111, 10.38775902, 10.47500863, 10.58734651]),
 'std_fit_time': array([0.48640388, 0.54188972, 0.32312826, 0.24714161, 0.14842868,
        0.32730394, 0.13539108, 0.41599338, 0.32063852, 0.29977776,
        0.28063302, 0.26952368, 0.29642975, 0.77294331, 0.60837693,
        0.13887996, 0.97207111, 0.32366657, 0.39782805, 0.12124233]),
 'mean_score_time': array([0.19659963, 0.23023028, 0.19785542, 0.23656998, 0.18900356,
        0.18304701, 0.18070879, 0.31864257, 0.18121219, 0.25439854,
        0.17940917, 0.18640585, 0.18420157, 0.18458905, 0.19900026,
        0.17820024, 0.18984222, 0.28306713, 0.18480706, 0.30592346]),
 'std_score_time': array([0.02078089, 0.07986323, 0.02787766, 0.0870642 , 0.00677907,
        0.00803078, 

In [28]:
results.best_score_

0.8237712502479553

In [29]:
results.best_params_

{'n_neurons1': 36, 'n_neurons2': 24}

In [30]:
results.refit_time_

12.938872575759888

In [31]:
results.n_splits_

5

In [32]:
Titanic_prediction_SVM=results.predict(TTestX)

C:\Users\Dara\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\keras\engine\sequential.py:331: RuntimeWarning: invalid value encountered in greater
  return (proba > 0.5).astype('int32')


In [33]:
TTestX.shape

(418, 47)

In [34]:
Titanic_prediction_SVM.shape

(418, 1)

In [35]:
T3['Survived']=Titanic_prediction_SVM

In [36]:
titanic_test.head(10)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
PassengerId,,,,,,,,,,,
892,3,Mr.,male,34.5,0,0,3,7.8292,C,Q,0
893,3,Mrs.,female,47.0,1,0,3,7.0000,C,S,1
894,2,Mr.,male,62.0,0,0,2,9.6875,C,Q,0
895,3,Mr.,male,27.0,0,0,3,8.6625,C,S,0
896,3,Mrs.,female,22.0,1,1,3,12.2875,C,S,1
897,3,Mr.,male,14.0,0,0,0,9.2250,C,S,0
898,3,Miss.,female,30.0,0,0,3,7.6292,C,Q,1
899,2,Mr.,male,26.0,1,1,2,29.0000,C,S,0
900,3,Mrs.,female,18.0,0,0,2,7.2292,C,C,1


In [37]:
prediction_df=T3[['Survived']]

In [38]:
prediction_df.to_csv('Downloads/Datascience/KaggleStuff/NN_prediction.csv')